# PuLP solvers comparison

This notebook compares the performance of different solvers in PuLP using a capacitated vehicle positioning problem as an example. The problem is formulated as a mixed-integer linear programming problem (MILP) and solved using three different solvers: PULP_CBC (COIN-OR Branch and Cut), GLPK (GNU Linear Programming Kit), and HiGHS (High-Performance Solver). The performance of the solvers is compared in terms of the solution time and the objective value.


In [1]:
# import watermark
%reload_ext watermark
%watermark

Last updated: 2024-03-18T18:27:05.581306+01:00

Python implementation: CPython
Python version       : 3.11.6
IPython version      : 8.22.2

Compiler    : MSC v.1935 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores   : 12
Architecture: 64bit



In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import pulp
import highspy

pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ox.settings.log_console=True
# ox.settings.use_cache=True
%watermark -w
%watermark -iv

Watermark: 2.4.3

pandas    : 2.2.1
numpy     : 1.26.4
pulp      : 2.8.0
matplotlib: 3.8.3
geopandas : 0.14.3
highspy   : 1.5.3



### Problem formulation:
- The objective function is to minimize the total response time of the police vehicles to events in their district.
- The constraints include 
    - a fixed number of police car locations
    - each event must be assigned to exactly one police car.
    - A police car can be assigned to an event only if it's placed at the corresponding location.
    - Each police car has a maximum capacity of events it can respond to (M).
- Complexity and Model Scalability: The presence of binary variables (x,y) transforms your problem into a Mixed-Integer Linear Programming (MILP) problem, inherently making it NP-hard. This complexity means that solving time can increase exponentially with the size of your data or the constraints of your problem

### Problem characteristics:
- The problem has 30 cars and over 2286 locations.
- The CostMatrix which holds the pairs of car nodes, event nodes and the cost of travel between them is a 30x2286 matrix (68580 elements). This can be considered a large-scale linear programming problem.
- The problem's density (ratio of non-zero entries in the cost matrix to the total possible entries) is usually sparse for road networks. Sparse problems have many zero or near-zero elements in their matrices, implying that not every police car location has a direct path to every event, which is typical in real-world road networks.

### Suggested methods:
Method often depends on the problem's specific characteristics, such as its size, density, and the nature of its constraints and variables.
- **Simplex Method**: Primarily used for solving linear programming problems, the Simplex method iterates through vertices of the feasible region in the direction of increasing value of the objective function until the optimum is reached. While incredibly efficient for linear problems, its performance may suffer as the problem scales or becomes more complex with integer constraints.

- **Interior-Point Methods**: These methods are more suited for large-scale linear programming problems and work by traversing the interior of the feasible region. They can be faster than the Simplex method for very large problems but might not be as efficient when dealing with MILP problems due to the necessity of handling binary or integer variables.

- **Branch and Bound (B&B)**: This is the most common approach for solving MILP problems. It involves solving a series of linear programming relaxations of the original problem, systematically dividing the problem (branching) into smaller subproblems, and eliminating regions that do not contain an optimal solution (bounding). This method is effective for problems with binary and integer variables but can be computationally expensive.

Of the above methods, the Branch and Bound method stands out as most suited for the given problem. All the solvers below (PULP_CBC_CMD, GLPK_CMD, and HiGHS), all can implement the B&B method effectively but with different performance characteristics and features.


### Formal definition of the problem:

#### Notation

Let's define the notation that will be used to formulate the problem:
- $N$ : The set of all nodes in the road network.
- $E$ : The set of events, each associated with a node in $N$ and a severity score.
- $P$ : The set of potential police car locations, a subset of $N$.
- $C_{ij}$ : The travel time from police car location $i \in P$ to event location $j \in E$.
- $x_i$ : Binary decision variable where $x_i = 1$ if a police car is positioned at node $i$, and $x_i = 0$ otherwise.
- $y_{ij}$ : Binary decision variable where $y_{ij} = 1$ if the event $j$ is assigned to the police car located at $i$, and $y_{ij} = 0$ otherwise.

**Objective Function**: minimize the total response time to all events. The response time is the travel time from a police car to an event, considering only the assignments where $y_{ij} = 1$.
$
\text{Minimize} \quad Z = \sum_{i \in P} C_{ij} \cdot y_{ij}
$

#### Constraints
1. **Police Car Placement Constraint**: Only $K$ police cars are available to be deployed.
$
\sum_{i \in P} x_i = K
$

2. **Event Assignment Constraint**: Each event must be assigned to exactly one police car.
$
\sum_{i \in P} y_{ij} = 1 \quad \forall j \in E
$

3. **Validity Constraint**: An event can only be assigned to a police car if that car is positioned at a node.
$
y_{ij} \leq x_i \quad \forall i \in P, \forall j \in E
$

4. **Capacity Constraint**: Each police car can only be assigned to a limited number of events.
$
\sum_{j \in E} y_{ij} \leq M_i \cdot x_i \quad \forall i \in P
$

5. **Non-Negativity and Integrality**: Ensure that the decision variables are binary.
$
x_i \in \{0, 1\} \quad \forall i \in P
$
$
y_{ij} \in \{0, 1\} \quad \forall i \in P, \forall j \in E
$

## 1. Data preparation

### 1.1 Importing and inspect the CostMatrix

We start by importing the `CostMatrix` that contains the travel time (cost) between each police car locations and the events. We also define the maximum number of events that each police car can respond to.

In [3]:
# import cost matrix from CSV and drop distance column as we only use travel time
CostMatrix = pd.read_csv("../data/ost/OstCostMatrix.csv")
CostMatrix.drop(columns="distance", inplace=True)
# CostMatrix.head(2)

# inspect the CostMatrix
print(f"Number of pairs: {len(CostMatrix)}")
print(f"CostMatrix statistics:")
print(CostMatrix.describe())

Number of pairs: 68580
CostMatrix statistics:
       travel_time  carNodeID  eventNodeID
count    68580.000  68580.000    68580.000
mean      3415.827  25998.033    21618.299
std       1775.315  13303.202    11914.866
min          0.000   2432.000      238.000
25%       1908.575  19224.000    11837.000
50%       3283.500  23805.000    22799.000
75%       4790.699  40160.000    30575.000
max       9930.399  46954.000    47703.000


### 1.2 Modify data structures for efficient computation

The `CostMatrix` is a 30x2286 matrix, which is a large-scale linear programming problem.  
As PuLP need to iterate over the decision variables to compute the objective function and constraints, we need to ensure that the data structures are efficient for this computation.

**Step 1**: Convert the `CostMatrix` into a nested dictionary for fast lookup of travel times. With keys as the police car locations and the events, and the values as the travel time between them. Note: since `carNode` and `eventNode` are being used as keys, they need to be unique, so we handle duplicates by averaging the travel times for those pairs.

**Step 2**: Reduction of problem size by removing the top 30% travel times. In those cases, there are probably other cars that are closer to the event, as there are multiple cars in the district.
E.g. the longest travel time could be traveling from the furthest east car position to the furthest west event position. But in these cases, closer cars will handle the event.


In [4]:
# Convert CostMatrix to Dictionary for Fast Lookups: {(carNodeID, eventNodeID): travel_time}
# Handling duplicates by averaging travel times for the same (carNodeID, eventNodeID) pairs
aggregated_cost_matrix = CostMatrix.groupby(['carNodeID', 'eventNodeID']).agg({'travel_time': 'mean'}).reset_index()

# Convert the aggregated CostMatrix DataFrame to a nested dictionary for fast lookup
cost_matrix_dict = aggregated_cost_matrix.set_index(['carNodeID', 'eventNodeID']).to_dict(orient='index')
cost_matrix_dict = {(i, j): v['travel_time'] for (i, j), v in cost_matrix_dict.items()}
# cost_matrix_dict

In [5]:
# Reduction of Problem Size
# Filter out the highest travel times, as there is probably a closer car to the event

# threshold to filter out pairs with the top X % highest travel times
DISCARD_THRESHOLD = 0.30
max_acceptable_travel_time = np.percentile(list(cost_matrix_dict.values()), 100 * (1 - DISCARD_THRESHOLD))

# Filter pairs exceeding the maximum acceptable travel time
filtered_pairs = {pair: time for pair, time in cost_matrix_dict.items() if time <= max_acceptable_travel_time}

# print statistics for the filtered_pairs
print(f"Filtering out {DISCARD_THRESHOLD*100:.0f}% highest travel times - keeping only travel times <= {max_acceptable_travel_time:.2f} sec")
print("Original nr of pairs: ", len(cost_matrix_dict), " | Filtered nr of pairs: ", len(filtered_pairs))
print("Original max travel time: ", np.max(list(cost_matrix_dict.values())), " | Filtered max travel time: ", np.max(list(filtered_pairs.values())))
print("Original min travel time: ", np.min(list(cost_matrix_dict.values())), " | Filtered min travel time: ", np.min(list(filtered_pairs.values())))

Filtering out 30% highest travel times - keeping only travel times <= 4492.76 sec
Original nr of pairs:  65970  | Filtered nr of pairs:  46179
Original max travel time:  9930.399  | Filtered max travel time:  4492.7
Original min travel time:  0.0  | Filtered min travel time:  0.0


Now, we have reduced the problem size and the CostMatrix is now in a dictionary format, which is more efficient for the solver to iterate over.

### 1.3 Setup LP problem

In [6]:
# Constants
K = 4  # Number of police car locations in final solution
M = 800   # Maximum number of events a single police car can respond to

# Sets
P = CostMatrix['carNodeID'].unique()  # Potential police car locations
E = CostMatrix['eventNodeID'].unique()  # Events

# Create the LP object - minimize total travel time
problem = pulp.LpProblem("PoliceCarLocationOptimization", pulp.LpMinimize) # Minimization problem
problem

PoliceCarLocationOptimization:
MINIMIZE
None
VARIABLES

### 1.4 Add decision variables and objective function

In [7]:
%%time
# Decision Variables
# x[i] = 1 if a police car is placed at location i, 0 otherwise
x = pulp.LpVariable.dicts("x", P, cat='Binary')  # Police car placement
# # y[i, j] = 1 if event j is assigned to police car i, 0 otherwise
y = pulp.LpVariable.dicts("y", filtered_pairs.keys(), cat='Binary')  # Event assignment

# Objective Function - minimize total response time
problem += pulp.lpSum([cost_matrix_dict[(i, j)] * y[(i, j)] for i in P for j in E if (i, j) in filtered_pairs]), "TotalResponseTime"
# This is the same as:
# for i in P:
#     for j in E:
#         if (i, j) in filtered_pairs:
#             problem += cost_matrix_dict[(i, j)] * y[(i, j)]
# And means that we want to minimize the total response time

CPU times: total: 297 ms
Wall time: 598 ms


### 1.5 Add problem constraints

In [8]:
%%time
# Police Car Placement Constraint
problem += pulp.lpSum([x[i] for i in P]) == K, "NumberOfPoliceCars"
# This is the same as:
# for i in P:
#     problem += x[i] <= 1
# And means that we can only place a police car at one location

# Event Assignment Constraint
for j in E:
    problem += pulp.lpSum([y[(i, j)] for i in P if (i, j) in filtered_pairs]) == 1, f"EventAssignment_{j}"
# This is the same as:
# for j in E:
#     problem += pulp.lpSum([y[(i, j)] for i in filtered_pairs if (i, j) in filtered_pairs]) == 1
# And means that each event must be assigned to exactly one police car

# Validity Constraint
for (i, j) in filtered_pairs:
    problem += y[(i, j)] <= x[i], f"Validity_{i}_{j}"
# This is the same as:
# for (i, j) in filtered_pairs:
#     problem += y[(i, j)] <= x[i]
# And means that we can only assign events to police cars that are placed

# Capacity Constraint
for i in P:
    problem += pulp.lpSum([y[(i, j)] for j in E if (i, j) in filtered_pairs]) <= M * x[i], f"Capacity_{i}"
# This is the same as:
# for i in P:
#     problem += pulp.lpSum([y[(i, j)] for j in E]) <= M * x[i]
# And means that each police car can only respond to a maximum of M events

CPU times: total: 734 ms
Wall time: 923 ms


## 2.0 Solvers

### 2.1 Solvers for testing

- **PULP_CBC (COIN-OR Branch and Cut)**
    - The default solver in PuLP. It is an open-source solver that is included in the PuLP package. It is a branch-and-cut solver that uses the COIN-OR CBC optimization engine.

- **GLPK (GNU Linear Programming Kit)**
    - An open-source solver that is included in the PuLP package. It is a mixed integer linear programming solver that uses the GNU Linear Programming Kit, which is a set of routines written in C and organized in the form of a callable library.

- **HiGHS (High-Performance Solver)**
    - A high-performance solver that use the HiGHS optimization engine. It has been designed to solve large-scale linear programming problems and is particularly well-suited for solving problems with a large number of constraints and variables. Its ability to utilize multiple threads can significantly speed up the solution process for complex problems. 

### 2.2 Solver configuration:
Configuration was set to balance accuracy against speed. These were the general settings, as each solver has its own specific parameters:
- **Mip: True** - signals solver to use algorithms and heuristics designed for integer constraints, ensuring that the solution adheres to the binary variable requirements.
    - However, mip=False can relax the integer constraints and simplify the solution space for optimization algorithms to navigate. Can sometimes be faster, but result might be invalid.
- **Presolve: True** - enables the solver to simplify the problem by removing redundant constraints and variables, which can significantly reduce the problem's complexity and solution time.
- **gapRel: 0.05** - sets the relative gap between the best solution found and the best possible solution. This parameter is used to terminate the solver when the relative gap is less than the specified value.
- **TimeLimit: 60** seconds - sets the maximum time the solver can run before terminating the solution process.

Will first run each solver wil the hypothetically optimal settings, then run them again with alternative settings to compare the performance of each solver.  

*Recall this is a minimization problem, so the lower the objective value, the better the solution.*

In [9]:
# confirm solvers available to PuLP
pulp.listSolvers(onlyAvailable=True)

['GLPK_CMD', 'PULP_CBC_CMD', 'HiGHS']


#### 2.3 PuLP CBC (COIN-OR Branch and Cut)

In [11]:
%%time
# CBC VERSION 1: Ideal settings
status = problem.solve(pulp.PULP_CBC_CMD(
    mip=True,  # mip=True since have binary variables. mip=False can sometimes be faster, but result might be invalid
    msg=False,  # Enable solver messages for more insight into the solving process
    # timeLimit=60,  # Limit the solver to 60 seconds
    gapRel=0.10,  # Accept a solution within 10% of the optimum
    presolve=True,  # Enable presolve to simplify the problem before solving
    warmStart=True,  # Use a warm start to speed up the solving process
    strong=True,  # Use a limited number of strong branching candidates for a balance between speed and accuracy
    # threads=4,  # Sets the maximum number of threads to be used by the solver (not working)
))

# inspect the status of the solution
print(f"status: {problem.status}, {pulp.LpStatus[problem.status]}")
print(f"Objective function value: {pulp.value(problem.objective):.2f} seconds, or {pulp.value(problem.objective)/60:.2f} minutes, or {pulp.value(problem.objective)/3600:.2f} hours")

status: 1, Optimal
Objective function value: 2050905.25 seconds, or 34181.75 minutes, or 569.70 hours
CPU times: total: 844 ms
Wall time: 1min 25s


In [85]:
%%time
# CBC VERSION 2: Alternative settings
status = problem.solve(pulp.PULP_CBC_CMD(
    mip=False,  # mip=True since have binary variables. mip=False can sometimes be faster, but result might be invalid
    msg=False,  # Enable solver messages for more insight into the solving process
    timeLimit=60,  # Limit the solver to 60 seconds
    # gapRel=0.1,  # Accept a solution within 10% of the optimum
    # presolve=True,  # Enable presolve to simplify the problem before solving
    # warmStart=True,  # Use a warm start to speed up the solving process
    # strong=True,  # Use a limited number of strong branching candidates for a balance between speed and accuracy
    # cuts=True,  # Enable all available cuts
    # threads=6,  # Sets the maximum number of threads to be used by the solver (not working)
))

# inspect the status of the solution
print(f"status: {problem.status}, {pulp.LpStatus[problem.status]}")
print(f"Objective function value: {pulp.value(problem.objective):.2f} seconds, or {pulp.value(problem.objective)/60:.2f} minutes, or {pulp.value(problem.objective)/3600:.2f} hours")

status: 1, Optimal
Objective function value: 2050905.25 seconds, or 34181.75 minutes, or 569.70 hours
CPU times: total: 938 ms
Wall time: 9.78 s


Observations:
- Original Settings: ``mip=True`` resulted in a solution, after 1min 25sec.
- Alternative Settings: ``mip=False`` resulted in a solution, after 10 seconds.

This is a good example of how relaxing the integer constraints can simplify the solution space for optimization algorithms to navigate, and can sometimes be faster. But we have to inspect that the result is valid acording to the problem's constraints. In this case, both solutions were the same valid solution.


#### 2.4 GLPK (GNU Linear Programming Kit)

In [21]:
%%time
# GLPK VERSION 1: Ideal settings
status = problem.solve(pulp.GLPK_CMD(
    mip=True,  # mip=True since have binary variables. mip=False can sometimes be faster, but result might be invalid
    msg=False,  # Display solver output for more insight
    # timeLimit=60,  # Solver time limit set to 60 seconds
    options=['--presol', '--cuts', '--mipgap', '0.1']
))

# inspect the status of the solution
print(f"status: {problem.status}, {pulp.LpStatus[problem.status]}")
print(f"Objective function value: {pulp.value(problem.objective):.2f} seconds, or {pulp.value(problem.objective)/60:.2f} minutes, or {pulp.value(problem.objective)/3600:.2f} hours")

status: 1, Optimal
Objective function value: 2050905.25 seconds, or 34181.75 minutes, or 569.70 hours
CPU times: total: 922 ms
Wall time: 1min 7s


In [75]:
%%time
# GLPK VERSION 2: Alternative settings
status = problem.solve(pulp.GLPK_CMD(
    mip=False,  # mip=True since have binary variables. mip=False can sometimes be faster, but result might be invalid
    msg=False,  # Display solver output for more insight
    timeLimit=60,  # Solver time limit set to 60 seconds
    # options=['--presol', '--cuts', '--mipgap', '0.1'] # presolve=True, cuts=True, mipgap=0.1
))

# inspect the status of the solution
print(f"status: {problem.status}, {pulp.LpStatus[problem.status]}")
print(f"Objective function value: {pulp.value(problem.objective):.2f} seconds, or {pulp.value(problem.objective)/60:.2f} minutes, or {pulp.value(problem.objective)/3600:.2f} hours")

status: 1, Optimal
Objective function value: 2050905.25 seconds, or 34181.75 minutes, or 569.70 hours
CPU times: total: 1.22 s
Wall time: 48.7 s


Observations:
- Original Settings: ``mip=True`` resulted in a solution, after 1min 7sec.
- Alternative Settings: ``mip=False`` resulted in a solution, after 50sec.

Again, relaxing the integer variable constraint resulted in reaching the same optimal solution faster.

#### 2.5 HiGHS (High-Performance Solver)

In [71]:
%%time

# HiGHS VERSION 1: Ideal settings
# status = problem.solve(pulp.HiGHS(mip=True, msg=False, options=['presolve', 'on'], timeLimit=60, gapRel=0.1, threads=12))
status = problem.solve(pulp.HiGHS(
    mip=True, # mip=True since have binary variables. mip=False can sometimes be faster, but result might be invalid
    msg=False, 
    # timeLimit=60,  # Solver time limit set to 60 seconds
    # threads=6,  # Adjust based on your system's CPU capabilities (not working)
    mip_rel_gap=9999999,
    parallel="on",
    presolve="on",
))

# inspect the status of the solution
print(f"status: {problem.status}, {pulp.LpStatus[problem.status]}")
print(f"Objective function value: {pulp.value(problem.objective):.2f} seconds, or {pulp.value(problem.objective)/60:.2f} minutes, or {pulp.value(problem.objective)/3600:.2f} hours")

status: 1, Optimal
Objective function value: 2050905.25 seconds, or 34181.75 minutes, or 569.70 hours
CPU times: total: 16.7 s
Wall time: 15.9 s


In [73]:
%%time
# HiGHS VERSION 2: Alternative settings
# status = problem.solve(pulp.HiGHS(mip=True, msg=False, options=['presolve', 'on'], timeLimit=60, gapRel=0.1, threads=12))
status = problem.solve(pulp.HiGHS(
    mip=False, # mip=True since have binary variables. mip=False can sometimes be faster, but result might be invalid
    msg=False,
    presolve="off",
    # options=[
    #     "presolve=on",  # Enable presolving to simplify the problem before solving
    #     "parallel=on",  # Enable parallel solving if supported
    #     "time_limit=60",  # Set a time limit to ensure the solver doesn't run indefinitely
    #     "simplex_dualise=on",  # Use the dual simplex algorithm for LP relaxation
    #     "simplex_crash=on",  # Use a crash algorithm to find a good starting basis for the simplex
    #     "simplex_strategy=6",  # Use a strategy that balances between speed and precision
    #     "mip_gap=0.1"  # Accept a solution within 5% of the optimal (relaxes accuracy for speed)
    # ],
    # threads=12  # Adjust based on your system's CPU capabilities (not working)
))

# inspect the status of the solution
print(f"status: {problem.status}, {pulp.LpStatus[problem.status]}")
print(f"Objective function value: {pulp.value(problem.objective):.2f} seconds, or {pulp.value(problem.objective)/60:.2f} minutes, or {pulp.value(problem.objective)/3600:.2f} hours")

status: 1, Optimal
Objective function value: 2050905.25 seconds, or 34181.75 minutes, or 569.70 hours
CPU times: total: 10.2 s
Wall time: 13.9 s


Observations:
- Original Settings: ``mip=True`` resulted in a solution, after 15sec.
- Alternative Settings: ``mip=False`` resulted in a solution, after 14.2sec. (13.9sec with ``presolve=off``)

Again, relaxing the integer variable constraint resulted in reaching the same optimal solution faster.

## 3.0 Conclusion

| Solver                    | Runtime (Integer Constraints) | Runtime (Relaxed Constraints) |
|---------------------------|-------------------------------------|--------------------------------------|
| PULP_CBC (COIN-OR Branch and Cut) | 1min 25sec                          | 10 seconds                           |
| GLPK (GNU Linear Programming Kit) | 1min 7sec                           | 50 seconds                           |
| HiGHS (High-Performance Solver)    | 15 seconds                          | 14.2 seconds                         |


In conclusion, the HiGHS solver seems like the fastest general solver. However, the other solvers can also be fast (or faster) if the integer constraints are relaxed. This shows that the choice of solver and its settings can have a significant impact on the solution time and the quality of the solution. It is important to experiment with different solvers and settings to find the best combination for a given problem.